# Lab 01: SoftwareIntro
The purpose of this homework is to guide you through opening the programs you've downloaded through Neurodebian and getting familiar with some of their basic functions.


## Preparation 

### Download example data

We will first download some practice MRI data from open science framework. While this is not a common outlet for scientists to share their MR imaging datasets, this is a useful repository for files that are larger than we would want to put on github and are in a simplified storage format for teaching and demonstrations like this one.

Download from osf: https://osf.io/bprq5/

After clicking on the link, you should see the download __lab01_images.tar.gz__ appear in your browser downloads. We will practice using the bash terminal to navigate to the Downloads folder, and to copy and unpack the download to our lab directory.


In [10]:
%%bash
#^the %% symbols tell the notebook to execute bash commands
cd ~/Downloads  #change directories to the Downloads folder, the ~ is shorthand for "home directory"
ls  #list the files there

In [12]:
%%bash
ls  #now osfshare is a directory, we will go into this directory to practice opening MR images

img
lab01_images.tar.gz
osfshare
SoftwareIntro.ipynb


In [16]:
%%bash
cd osfshare
ls  #list contents

sub-97-T1w_brain_mask.nii.gz
sub-97-T1w_brain.nii.gz
sub-97-T1w_defaced.nii.gz
sub-97_task-flanker_bold.nii.gz


### Preview the data directly in jupyter

We'll first get used to terms about the spatial orientation of an MR image. We'll use a viewer that is part of nipype, which is a collection of neuroimaging tools written in the python programming language. 

More information about the NiftiWidget can be found here: http://nipy.org/niwidgets/examples.html. 

__Import an example image that comes with the package__

In [40]:
#import the widget from the niwidgets package
from niwidgets import NiftiWidget   

#import example data that comes with niwidgets
from niwidgets import examplet1   

#commands that open the image in the widget
#test_widget = NiftiWidget(examplet1)
test_widget.nifti_plotter()

interactive(children=(IntSlider(value=45, continuous_update=False, description='x', max=90), IntSlider(value=54, continuous_update=False, description='y', max=108), IntSlider(value=45, continuous_update=False, description='z', max=90), Dropdown(description='colormap', options=('viridis', 'Accent', 'Blues', 'BrBG', 'BuGn', 'BuPu', 'CMRmap', 'Dark2', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'Paired', 'Pastel1', 'Pastel2', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Set1', 'Set2', 'Set3', 'Spectral', 'Vega10', 'Vega20', 'Vega20b', 'Vega20c', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg', 'bwr', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar', 'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'jet', 'nipy_spectral', 'ocean', 'pink', 'prism', 'rainbow', 'seismic', 'spectral', 'spring', 'summer', 'tab10', 'tab20', 'tab20b', 'tab20c', 'terrain', 'winter'), value='viridis'), Output()), _dom_classes=('widget-interact',))

__Import our own image__ <br>
What are some similarities and differences?

In [42]:
#give our anatomical image a variable name
mydata='./osfshare/sub-97-T1w_defaced.nii.gz'

#what exactly does each do below
#mydata_widget = NiftiWidget(mydata)
mydata_widget.nifti_plotter()

interactive(children=(IntSlider(value=119, continuous_update=False, description='x', max=239), IntSlider(value=127, continuous_update=False, description='y', max=255), IntSlider(value=127, continuous_update=False, description='z', max=255), Dropdown(description='colormap', options=('viridis', 'Accent', 'Blues', 'BrBG', 'BuGn', 'BuPu', 'CMRmap', 'Dark2', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'Paired', 'Pastel1', 'Pastel2', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Set1', 'Set2', 'Set3', 'Spectral', 'Vega10', 'Vega20', 'Vega20b', 'Vega20c', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg', 'bwr', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar', 'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'jet', 'nipy_spectral', 'ocean', 'pink', 'prism', 'rainbow', 'seismic', 'spectral', 'spring', 'summer', 'tab10', 'tab20', 'tab20b', 'tab20c', 'terrain', 'winter'), value='viridis'), Output()), _dom_classes=('widget-interact',))

The images we have been viewing are high-resolution images using [T1 contrast](http://www.med.harvard.edu/aanlib/basicsMR.html), and the image format is [nifti](https://brainder.org/2012/09/23/the-nifti-file-format/). As you see above, the image is 3-dimensional and can be opened in 3 orthogonal views simultaneously; coronal, sagittal and axial views.The viewer above is useful for getting familiar with the x,y,and z "planes" of the iamge. Every 3-dimensional pixel in the image (called a voxel) has a unique x,y,z coordinate.  <br>
<br>
It is also common to refer to different parts of the image by their location in neuroanatomical terms: L, R, A, P, S, I (Left, Right, Anterior=front, Posterior=back, Superior=top, Inferior=bottom). We will see that some viewers will show us these labels and "Left" in the image is not always the "Left" part of the head! <br>
<br>
It may be obvious, but also note that in these images we cannot see at the level of single neurons. The anatomy we focus on with MRI is from a "macroscopic" view. 


## FSL package viewer

A very common neuroimaging package is FSL. FSL has viewer that we can open in the bash terminal, and it has some very helpful features for understanding more about images.

__We'll first see how to:__ <br>
<br>
__(a) learn basic information about the image attributes__ <br> 
-What kind of image is this <br>
-What is the voxel size <br>
-Others? <br>
<br>
__(b) look at the image histogram.__ <br>
An image histogram shows the distribution of all the intensity values that make up the image. Looking at these is helpful to see what intensities are generally “brain” versus background and to get comfortable with the idea that images are a collection of voxels with different intensity values (e.g., see Handbook Figure 2.1, p14). The intensity values mean something about what tissue we *think* we’re seeing based on the physics of MRI. It is also helpful to remember that the basis of most image processing tools is signal processing and statistics on these intensity values to improve the detection of the "signal" we're interested in (e.g., brain structure or activity) from noise (e.g., motion). <br>
-What is the range of values in our sub-97-T1w_defaced.nii.gz image <br>
-Is the histogram multi-modal? What would that tell us?  <br>
-What are the typical values for gray matter, white matter, and CSF? <br>
<br>
__(c) look at our functional image and its timeseries.__ <br>
-Compare and contrast the image intensities of white matter and gray matter from the structural with the functional image. <br>
-Show the image timeseries <br>
-Show the movie of volumes over time, how many dimensions is the file? <br>
-Compare the spatial resolution of the structural and functional <br>
-Can we overlay the functional on top of the structural image? <br>

In [43]:
%%bash

#the viewer version we'll use is called fslview, you can open it without opening an image by typing fslview
fslview


## Problem 4

The black lines represent the time series data from the grid of voxels within the green square cursor at the center of the cross-hair in the brain viewer window. To review, fMRI data are a set of intensity values for 3D images of the brain that are collected over time. For instance, imagine the structural image above (and associated intensity values you see in the magnified matrix) now collected multiple times over the course of 5 minutes. The result is each voxel has not only one intensity value, rather each voxel now has a timeseries of intensity values. In functional MRI these intensity values reflect the BOLD response, which we'll talk about more next week. For now, it may be helpful to see this idea visualized again from the Aguirre 2014 paper we read:

MV: below is too complex for the first lab; our goal for this one is to have them open a functional image in a viewer and play around with seeing the resolution and udnerstanding what a timeseries looks like from different voxels. This could be problem 3. I like afni viewer for this because it introduces them to a common viewer and the grid is nice for learning how the timeseries changes as you move around the brain.

Then have problem 4 be a simple coding challenge to get warmed up on basic commands we will use a lot. May include a problem for setting up their github configuration to their username on their virtual machine.

In [ ]:
%%bash
afni /home/brain/nilearn_data/haxby2001/subj2/bold.nii.gz &

In [ ]:
# mv suggests we hold this after we talk about what masks are 

import nibabel as nib
from nilearn import input_data
# load the face mask from the haxby dataset into python
face_mask_img = nib.load(haxby_dataset.mask_face[0])
# load the house mask from the haxby dataset into python
house_mask_img = nib.load(haxby_dataset.mask_house[0])
# concatonate the masks to make 4D image (each 3D image has one mask)
haxby_maps = nib.funcs.concat_images([face_mask_img, house_mask_img])

# nilearn utility to apply the masks to a dataset
masker = input_data.NiftiMapsMasker(
    haxby_maps, resampling_target="data", detrend=True, memory='nilearn_cache', memory_level=10, verbose=2)
# get the timeseries 
time_series = masker.fit_transform(func_filename)

In [ ]:
%matplotlib inline
haxby_dataset
import matplotlib.pyplot as plt

# plot the two time series
face_plot, = plt.plot(time_series[:,0], label='face')
house_plot, = plt.plot(time_series[:,1], label='house')
plt.legend([face_plot, house_plot], ['face', 'house'])
plt.show()
